In [ ]:
import pandas as pd
import numpy as np
import statistics as st
import scipy.stats
import matplotlib.pyplot as plt
from collections import Counter

## Load data

In [ ]:
deployments = pd.read_csv('deployments.csv')
proposals = pd.read_csv('proposals.csv', dtype={4 : 'float'})
votes = pd.read_csv('votes.csv', dtype={5 : 'float'})

We filter to remove deployments with no proposals or voters. The initial amount of them is:

In [ ]:
print(len(deployments), 'deployments')
print(len(proposals), 'proposals')
print(len(votes), 'votes')

We eliminate deployments without proposals

In [ ]:
#Votes and proposals should not change, as we are removing deployments that do not have proposals.
no_props = deployments[deployments['proposals_count'] == 0]['deployment_id'].to_list()

deployments = deployments[~deployments['deployment_id'].isin(no_props)]
proposals = proposals[~proposals['deployment_id'].isin(no_props)]
votes = votes[~votes['deployment_id'].isin(no_props)]

print(len(deployments), 'deployments')
print(len(proposals), 'proposals (should not change)')
print(len(votes), 'votes (should not change)')

We remove deployments with < 2 voters

In [ ]:
minVoters = 2

In [ ]:
no_voters = deployments[deployments['unique_voters'] < minVoters]['deployment_id'].to_list()

deployments = deployments[~deployments['deployment_id'].isin(no_voters)]
proposals = proposals[~proposals['deployment_id'].isin(no_voters)]
votes = votes[~votes['deployment_id'].isin(no_voters)]

print(len(deployments), 'deployments now')
print(len(proposals), 'proposals now')
print(len(votes), 'votes now')

### Save datasets


In [ ]:
deployments.to_csv('deployments_filtered.csv', index=False)
proposals.to_csv('proposals_filtered.csv', index=False)
votes.to_csv('votesFinal_filtered.csv', index=False)

# Separate and analyze DF based on size (members)

In [ ]:
#Create df based on size

deployments10 = deployments[deployments['unique_voters']<=10] 
deployments100 = deployments[np.logical_and(deployments['unique_voters']<=100, deployments['unique_voters']>10)] 
deployments1k = deployments[np.logical_and(deployments['unique_voters']<=1000, deployments['unique_voters']>100)] 
deployments10k = deployments[np.logical_and(deployments['unique_voters']<=10000, deployments['unique_voters']>1000)] 
deployments100k = deployments[np.logical_and(deployments['unique_voters']<=100000, deployments['unique_voters']>10000)] 
deployments1M = deployments[np.logical_and(deployments['unique_voters']<=1000000, deployments['unique_voters']>100000)]
deployments10M = deployments[deployments['unique_voters']>1000000] 

We show the total number of deployments for each size.

In [ ]:
platform = deployments.groupby('platform')['deployment_id'].count()
# platform = deployments10.groupby('platform')['deployment_id'].count()
# platform = deployments100.groupby('platform')['deployment_id'].count()
# platform = deployments1k.groupby('platform')['deployment_id'].count()
# platform = deployments10k.groupby('platform')['deployment_id'].count()
# platform = deployments100k.groupby('platform')['deployment_id'].count()
# platform = deployments1M.groupby('platform')['deployment_id'].count()
# platform = deployments10M.groupby('platform')['deployment_id'].count()

print(platform)
print('Total deployments', platform.sum())

We show voters by platform

In [ ]:
votantes = deployments.groupby('platform')['unique_voters']

print('Total ', votantes.sum())
print('------------------------')
print('Min', votantes.min())
print('------------------------')
print('Max', votantes.max())
print('------------------------')
print('Mean', votantes.mean())
print('------------------------')
print('Median', votantes.median())

Unique voters by deployment

In [ ]:
for data in [("total", deployments), ("10", deployments10), ("100", deployments100), ("1k", deployments1k), ("10k", deployments10k), ("100k", deployments100k), ("1M", deployments1M), ("10M", deployments10M)]:
    print('----- Size analysis:', data[0], '-----')
    print('The min is:', min(data[1]['unique_voters']))
    print('The mean is:', data[1]['unique_voters'].mean())
    print('The median is:', data[1]['unique_voters'].median())
    print('The max is:', max(data[1]['unique_voters']))
    print('------------------------')

Count of proposals

In [ ]:
for data in [("total", deployments), ("10", deployments10), ("100", deployments100), ("1k", deployments1k), ("10k", deployments10k), ("100k", deployments100k), ("1M", deployments1M), ("10M", deployments10M)]:
    print('----- Size analysis:', data[0], '-----')
    print('The min is:', min(data[1]['proposals_count']))
    print('The mean is:', data[1]['proposals_count'].mean())
    print('The median is:', data[1]['proposals_count'].median())
    print('The max is:', max(data[1]['proposals_count']))
    print('------------------------')

Total number of votes

In [ ]:
for data in [("total", deployments), ("10", deployments10), ("100", deployments100), ("1k", deployments1k), ("10k", deployments10k), ("100k", deployments100k), ("1M", deployments1M), ("10M", deployments10M)]:
    print('----- Size analysis:', data[0], '-----')
    print('The min is:', min(data[1]['votes_count']))
    print('The mean is:', data[1]['votes_count'].mean())
    print('The median is:', data[1]['votes_count'].median())
    print('The max is:', max(data[1]['votes_count']))
    print('------------------------')

# Proposals a voter participates in
Votes per voter as a percentage of total proposals
E.g., a voter votes on average on 15% of DAO proposals

Calculation:
1) Calculate the average within the community
2) Calculate the average of averages

In [ ]:
votos = votes.groupby(['deployment_id', 'voter'])['vote_id'].count()

In [ ]:
# Change to use other desployment group
ids = deployments100k['deployment_id'] 

In [ ]:
avgVotesByVoter = []

for deployment in ids:
    num_props = int(deployments[deployments['deployment_id']==deployment]['proposals_count'])    
    a = votos[deployment].mean() / num_props * 100
    avgVotesByVoter.append(a)  

print('The minimum is', min(avgVotesByVoter), '% props per voter')       
print('The mean is', st.mean(avgVotesByVoter), '% props per voter')
print('The median is', st.median(avgVotesByVoter), '% props per voter')
print('The maximum is', max(avgVotesByVoter), '% props per voter')

# Votes per proposal / Voter turnout

In [ ]:
# Change to use other desployment group
deployment_group = deployments10M 

In [ ]:
i=0
votosPorPropuesta = []

props = proposals[proposals['deployment_id'].isin(set(deployment_group['deployment_id']))]['proposal_id']

                                                
for proposal in props:
    num_votos = int(proposals[proposals['proposal_id']==proposal]['votes_count'])
    dep_id = proposals[proposals['proposal_id']==proposal]['deployment_id'].to_list()[0]
    votantes = int(deployments[deployments['deployment_id']==dep_id]['unique_voters'])
    votosPorPropuesta.append(num_votos / votantes * 100) 

       
#Calculate min, max, median, mean of votes per proposal
print('The minimum is', min(votosPorPropuesta), '% votes per prop')
print('The mean is', st.mean(votosPorPropuesta), '% votes per prop')
print('The median is', st.median(votosPorPropuesta), '% votes per prop')
print('The maximum is', max(votosPorPropuesta), '% votes per prop')

### VP per proposal

In [ ]:
#vp by proposal
#percentages
join = proposals.merge(deployments, how='inner', on='deployment_id')
join['estimated_vp'] = join['estimated_vp'].astype(float)


#remove all rows containing nan
print(len(join))
join = join.dropna(subset=['estimated_vp'])
join = join.dropna(subset=['totalVP'])
print(len(join))

#remove estimated_vp that are 0
join = join[join['estimated_vp']>0]

vpPorPropuesta = (join['totalVP'] / join['estimated_vp'] * 100).to_list()

print('The mean is', st.mean(vpPorPropuesta))
print('The median is', st.median(vpPorPropuesta))
print('The min is', min(vpPorPropuesta))
print('The max is', max(vpPorPropuesta))

# Median VP per voter and deployment 

In [ ]:
# Change to use other desployment group
deployment_group = deployments10M 

In [ ]:
unionVotosDep = votes.merge(deployment_group, how='inner', on='deployment_id')

#unionVotosDep = unionVotosDep[['id', 'deployment_id', 'voter', 'weight', 'estimated_vp']]

#Remove all rows containing 'nan' (after cleaning, none)
unionVotosDep = unionVotosDep.dropna(subset=['estimated_vp'])
unionVotosDep = unionVotosDep.dropna(subset=['weight'])
#Remove estimatedVP = 0
unionVotosDep['estimated_vp'] = unionVotosDep['estimated_vp'].apply(lambda x: 0.000000000000001 if x<=0 else x)

VP_Votante = unionVotosDep.groupby(['deployment_id', 'voter', 'estimated_vp'])['weight'].agg(median='median').reset_index()
VP_Votante['porc'] = VP_Votante['median'] / VP_Votante['estimated_vp'] * 100

print('The min is', min(VP_Votante['porc']), '% vp per voter')
print('The mean is', st.mean(VP_Votante['porc']), '% vp per voter')
print('The median is', st.median(VP_Votante['porc']), '% vp per voter')
print('The max is', max(VP_Votante['porc']), '% vp per voter')

# Gini -> Inequality of effective VP distribution

In [ ]:
#gini 

def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq: http://www.statsdirect.com/help/content/image/stat0206_wmf.gif
    # from: http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array += 1e-9 #values cannot be 0
    array = np.sort(array) #values must
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0]#number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient

In [ ]:
VP_voter_dep = votes.groupby(["deployment_id", "voter"])['weight'].agg(median='median').reset_index()

ginis = VP_voter_dep.groupby('deployment_id', group_keys=True).apply(lambda x: gini(np.array(x['median'] ))).reset_index(name='gini')

#Negative Ginis by approximation. We replace them with 0.
ginis['gini'] = ginis['gini'].apply(lambda x: 0 if x<0 else x)

print('The mean is', st.mean(ginis['gini']))
print('The median is', st.median(ginis['gini']))
print('The min is', min(ginis['gini']))
print('The max is', max(ginis['gini']))


In [ ]:
#Filtering by size

#Change to use other desployment group
deployment_group = deployments10M 

In [ ]:
#Filtering
votos_deps = votes[votes['deployment_id'].isin(deployment_group['deployment_id'])]

VP_voter_dep = votos_deps.groupby(["deployment_id", "voter"])['weight'].agg(median='median').reset_index()

ginis = VP_voter_dep.groupby('deployment_id', group_keys=True).apply(lambda x: gini(np.array(x['median'] ))).reset_index(name='gini')

ginis['gini'] = ginis['gini'].apply(lambda x: 0 if x<0 else x)

print('The mean is', st.mean(ginis['gini']))
print('The median is', st.median(ginis['gini']))
print('The min is', min(ginis['gini']))
print('The max is', max(ginis['gini']))

### Voters who accumulate together more than 50% of the voting power for each deployment

In [ ]:
power = votes.groupby(["deployment_id", "voter"])['weight'].agg(median='median').reset_index().sort_values(["deployment_id", "median"],ascending=False)
power


In [ ]:
# Change to use other desployment group
deployment_group = deployments10M 

In [ ]:
result2 = []
dict_res2 = {}

for dep in deployment_group['deployment_id']:

    depVP = deployments[deployments['deployment_id'] == dep]['estimated_vp'].to_list()[0]
    if depVP == 0:
        dict_res2[dep] = None
        continue
    acum = np.cumsum(np.array(power[power['deployment_id']==dep]['median']))
    acum = acum[acum <= depVP/2]
    r = len(acum) + 1
    voters = int(deployments[deployments['deployment_id'] == dep]['unique_voters'])
    total = r / voters * 100
    result2.append(total) 
    dict_res2[dep] = total
    
print('The mean is', st.mean(result2))
print('The median is', st.median(result2))
print('The min is', min(result2))
print('The max is', max(result2))

In [ ]:
#with >=50% instead of >50%

result2 = []
dict_res2 = {}

for dep in deployment_group['deployment_id']:
    depVP = deployments[deployments['deployment_id'] == dep]['estimated_vp'].to_list()[0]
    if depVP == 0:
        dict_res2[dep] = None
        continue
    acum = np.cumsum(np.array(power[power['deployment_id']==dep]['median']))
    acum = acum[acum < depVP/2]
    r = len(acum) + 1
    voters = int(deployments[deployments['deployment_id'] == dep]['unique_voters'])
    total = r / voters * 100
    result2.append(total) 
    dict_res2[dep] = total

print('The mean is', st.mean(result2))
print('The median is', st.median(result2))
print('The min is', min(result2))
print('The max is', max(result2))

# Age of deployments

In [ ]:
from datetime import datetime

#group the proposals by deployment
deps = proposals.groupby('deployment_id')['date'].agg(['min','max'])
deps['min'] = deps['min'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
deps['max'] = deps['max'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
deps['age'] = deps['max']-deps['min']

deployments = deployments.merge(deps, on='deployment_id', how='inner')

Observe if the last vote is more than 3 months old and more than 6 months old since the data retrieval date.

In [ ]:
#step 1: create the date of retrieval
date_str = "9 August 2023"
captura = datetime.strptime(date_str, "%d %B %Y")

print(captura)

#step 2: for every deployment, captura date - max date
df = pd.DataFrame(deployments)
#df['vivas'] = (captura - deployments['max']).dt.days
df['alive'] = ((captura - deployments['max']) / pd.Timedelta(days=30.44)).astype(int)

print(df.tail())

print("Alive in the last 3 months")
df3 = df[df['alive']<=3]
df3 = df3.groupby('platform').count()
print(df3)

print("Alive in the last 6 months")
df6 = df[df['alive']<=6]
df6 = df6.groupby('platform').count()
print(df6)


Create new column in dataset to know how much time has passed since the last vote and the retrieval date

In [ ]:
from datetime import datetime

date_str = "9 August 2023"
captura = datetime.strptime(date_str, "%d %B %Y")

df = pd.DataFrame(deployments)
df['tiempo_sin_actividad'] = ((captura - deployments['max']) / pd.Timedelta(days=30.44)).astype(int)

The following are with no proposals in the last 6 months

In [ ]:
print('<=10')
deps = df[df['deployment_id'].isin(deployments10['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

print('<=100')
deps = df[df['deployment_id'].isin(deployments100['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

print('<=1k')
deps = df[df['deployment_id'].isin(deployments1k['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

print('<=10k')
deps = df[df['deployment_id'].isin(deployments10k['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

print('<=100k')
deps = df[df['deployment_id'].isin(deployments100k['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

print('<=1m')
deps = df[df['deployment_id'].isin(deployments1M['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

print('<=10m')
deps = df[df['deployment_id'].isin(deployments10M['deployment_id'])]
print((deps['tiempo_sin_actividad']>6).sum(),',', (deps['tiempo_sin_actividad']>6).sum()/len(deps)*100, '%')
print('----------------')

Number of DAOs according its activity in the last 6 months

In [ ]:
activeDAOs = deployments[deployments['tiempo_sin_actividad']<=6]
nonActiveDAOs = deployments[deployments['tiempo_sin_actividad']>6]
print(len(deployments))
print(len(activeDAOs))
print(len(nonActiveDAOs))

Oldest and newest deployment

In [ ]:
print(min(deployments['edad']))
print(max(deployments['edad']))

Minimum, average, median, and maximum age data for each DAO platform

In [ ]:
print('aragon')
print(min(deployments[deployments['platform']=='aragon']['edad']))
print(deployments[deployments['platform']=='aragon']['edad'].mean())
print(deployments[deployments['platform']=='aragon']['edad'].median())
print(max(deployments[deployments['platform']=='aragon']['edad']))
print('----------------')

print('daohaus')
print(min(deployments[deployments['platform']=='daohaus']['edad']))
print(deployments[deployments['platform']=='daohaus']['edad'].mean())
print(deployments[deployments['platform']=='daohaus']['edad'].median())
print(max(deployments[deployments['platform']=='daohaus']['edad']))
print('----------------')

print('daostack')
print(min(deployments[deployments['platform']=='daostack']['edad']))
print(deployments[deployments['platform']=='daostack']['edad'].mean())
print(deployments[deployments['platform']=='daostack']['edad'].median())
print(max(deployments[deployments['platform']=='daostack']['edad']))
print('----------------')

print('governor')
print(min(deployments[deployments['platform']=='governor']['edad']))
print(deployments[deployments['platform']=='governor']['edad'].mean())
print(deployments[deployments['platform']=='governor']['edad'].median())
print(max(deployments[deployments['platform']=='governor']['edad']))
print('----------------')

print('realms')
print(min(deployments[deployments['platform']=='realms']['edad']))
print(deployments[deployments['platform']=='realms']['edad'].mean())
print(deployments[deployments['platform']=='realms']['edad'].median())
print(max(deployments[deployments['platform']=='realms']['edad']))
print('----------------')

print('snapshot')
print(min(deployments[deployments['platform']=='snapshot']['edad']))
print(deployments[deployments['platform']=='snapshot']['edad'].mean())
print(deployments[deployments['platform']=='snapshot']['edad'].median())
print(max(deployments[deployments['platform']=='snapshot']['edad']))
print('----------------')

print('tally')
print(min(deployments[deployments['platform']=='tally']['edad']))
print(deployments[deployments['platform']=='tally']['edad'].mean())
print(deployments[deployments['platform']=='tally']['edad'].median())
print(max(deployments[deployments['platform']=='tally']['edad']))


Metric by deployments size.

In [ ]:
# Change to use other desployment group

print('<=10')
print(min(deployments10['edad']))
print(deployments10['edad'].mean())
print(deployments10['edad'].median())
print(max(deployments10['edad']))
